In [1]:
import nltk
import numpy as np
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

from src.dataset_utils import MotleyFoolDataset

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\damie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\damie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\damie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\damie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Load and prepare dataset

Should be `(1141, 3821, 33174)`

In [3]:
dataset = MotleyFoolDataset('TMF_dataset_annotated.zip')
test_set = dataset['Q1 2023':'Q4 2023']
val_set = dataset['Q1 2022':'Q4 2022']
train_set = dataset[:'Q4 2021']

len(test_set), len(val_set), len(train_set)

(1141, 3821, 33174)

In [ ]:
X_train_val = [instance['content'] for instance in dataset[:'Q4 2022']]
y_train_val = [instance['direction'] == 'UP' for instance in dataset[:'Q4 2022']]

X_test = [instance['content'] for instance in test_set]
y_test = [instance['direction'] == 'UP' for instance in test_set]

# Utilities for the grid search

In [ ]:
### From: https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

stopwords_set = set(stopwords.words('english'))  # set for appartenance check optim
lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer("english")

def lemm_tokenizer(x):
  x = word_tokenize(x)
  tags = nltk.pos_tag(x)
  return [
      lemmatizer.lemmatize(word, get_wordnet_pos(tag))
      for word, tag in tags
      if word not in stopwords_set
  ]

def porter_stem_tokenizer(x):
  x = word_tokenize(x)
  return [porter_stemmer.stem(word) for word in x if word not in stopwords_set]

def snowball_stem_tokenizer(x):
  x = word_tokenize(x)
  return [snowball_stemmer.stem(word) for word in x if word not in stopwords_set]

In [1]:
vect_parameter_grid = {
    "vect__tokenizer": (
        lemm_tokenizer,
        porter_stem_tokenizer,
        snowball_stem_tokenizer,
    ),
    "vect__max_df": (0.2, 0.5, 0.8, 1.0),
    "vect__ngram_range": ((1, 1), (1, 2), (2, 2)),  # unigrams or bigrams
    "vect__norm": ("l1", "l2", None),
}

vectorizer_factory = lambda: ("vect", TfidfVectorizer(
    stop_words=None,
    token_pattern=None, # will not be used since 'tokenizer' is not None
))

def do_grid_search(parameter_grid, classifier=None, pipeline=None):
  parameter_grid |= vect_parameter_grid

  if pipeline is None:
    pipeline = Pipeline([
      vectorizer_factory(),
      ("clf", classifier),
    ])

  grid_search = GridSearchCV(
      estimator=pipeline,
      scoring='accuracy',
      param_grid=parameter_grid,
      #n_jobs=1, # Cannot use multiple processes due to tokenizer
      cv=[  # Remove cross validation and use our pre-made split
          # An iterable yielding (train, test) splits as arrays of indices.
          (np.arange(*dataset.range(q_stop='Q4 2021')),
           np.arange(*dataset.range('Q1 2022', 'Q4 2022')))
      ]
  )

  grid_search.fit(X_train_val, y_train_val)

  print('Best parameters:')
  best_parameters = grid_search.best_estimator_.get_params()
  for param_name in sorted(parameter_grid.keys()):
      print(f"{param_name}: {best_parameters[param_name]}")


  y_pred = grid_search.predict(X_test)
  ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

  print('Accuracy score:', accuracy_score(y_test, y_pred))
  print(classification_report(y_test, y_pred)) 

  return grid_search

NameError: name 'lemm_tokenizer' is not defined

# Models

## Naive Bayes

In [ ]:
do_grid_search(
    {"clf__alpha": np.logspace(-6, 6, 13)},
    MultinomialNB()
)

## Logistic Regression

In [ ]:
do_grid_search(
    {
        "clf__penalty": ('l1', 'l2', 'elasticnet', None),
        "clf__C": np.logspace(-6, 6, 13),
    },
    LogisticRegression()
)

## Support Vector Machine

### Linear SVC

In [ ]:
do_grid_search(
    {"clf__C": np.logspace(-6, 6, 13)},
    pipeline=Pipeline([
      vectorizer_factory(),
      ("lsa", TruncatedSVD(n_components=100)),
      ("norm", Normalizer()),
      ("clf", SVC(kernel='linear')) # So that it still is a linear classifier
    ])
)